<a href="https://colab.research.google.com/github/SuperSpawn/Side-Projects/blob/main/MeNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
1 + 5

6

In [30]:
class Stat:
  name: str
  value: float

  def __init__(self, name, value):
    self.name = name
    self.value = value

  def __str__(self):
    return f"{self.name} - {self.value}"

class Character:
  def __init__(self, name, stats) -> None:
    self.name = name
    self.stats = stats

  def __str__(self) :
    stats_str = ", ".join([str(stat) for stat in self.stats])
    return f"{self.name} - [{stats_str}]"

In [42]:
actions = ['eat', 'sleep', 'meet someone new', 'drink water']

In [31]:
import torch
from torch import nn
import torch.optim as optim

# Define the network
class MeNN(nn.Module):
    def __init__(self, input_size, middle_size, output_size):
        super(MeNN, self).__init__()
        self.fc1 = nn.Linear(input_size, middle_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(middle_size, output_size)
        self.double()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [7]:
stats = [Stat('hungry', 0.5), Stat('tired', 0.9), Stat('shy', 0.1)]
max = Character('Max', stats)
print(max)

Max - [hungry - 0.5, tired - 0.9, shy - 0.1]


In [22]:
# Get data
# Train the model
# Test the model

In [8]:
import csv

def save_dataset(output_file, data):
  with open(output_file, "w", newline="") as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=["input", "output"])

    # Write the header row
    writer.writeheader()

    # Write each data point as a row in the CSV file
    for item in data:
        writer.writerow(item)



In [32]:
# Create data

# Generate data
# save it to a file
# load file for training
import random
import numpy as np

def generate_instance(ranges):
  instance = []
  for inp in ranges:
    val = random.uniform(inp[0], inp[1])
    instance.append(val)
  return instance

def generate_dataset_case(input_ranges, output_ranges, size):
    dataset = []
    for _ in range(size):
      data = []
      data.append(generate_instance(input_ranges))
      data.append(generate_instance(output_ranges))
      dataset.append(data)
    return dataset

def generate_dataset(size, flags, case_size):
  total_data = []
  for i in range(size):
    input_ranges = np.full((size, 2), (0.5, 0.5))
    output_ranges = np.full((size, 2), (0.5, 0.5))

    input_ranges_low = input_ranges.copy()
    input_ranges_high = input_ranges.copy()
    output_ranges_low = output_ranges.copy()
    output_ranges_high = output_ranges.copy()

    input_ranges_low[i] = (0.0, 0.5)
    input_ranges_high[i] = (0.5, 1.0)

    output_ranges_low[i] = (0.0, 0.5)
    output_ranges_high[i] = (0.5, 1.0)

    if flags[i]['reverse']:
      temp = output_ranges_low
      output_ranges_low = output_ranges_high
      output_ranges_high = temp

    dataset_case_low = generate_dataset_case(input_ranges_low, output_ranges_low, case_size)
    dataset_case_high = generate_dataset_case(input_ranges_high, output_ranges_high, case_size)

    dataset_case = [i for i in dataset_case_low] + dataset_case_high

    total_data = [i for i in dataset_case] + total_data
  return total_data

dataset = generate_dataset(4, [{'reverse': False}, {'reverse': False}, {'reverse': True}, {'reverse': False}], 500)
print(len(dataset))
# print(dataset)
# save_dataset('test.csv', dataset)

4000


In [2]:
import torch
import torch.nn as nn

ce = nn.CrossEntropyLoss()

logits = torch.tensor([[-1.90, -0.29, -2.30], [-0.29, -1.90, -2.30]])
target = torch.tensor([[0., 1., 0.], [1., 0., 0.]])
print("Cross entropy: %.3f" % ce(logits, target))

Cross entropy: 0.288


In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
import csv
import numpy as np

class MeNNDataset(Dataset):
    def __init__(self, data):
        self.data = np.array(data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return a single data point and its label
        return self.data[idx]



In [31]:
testset = None

In [34]:
# Create datasets and data loaders
train_dataset = MeNNDataset(dataset)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataset = MeNNDataset("test.csv")
# test_loader = DataLoader(test_dataset, batch_size=32)

In [48]:
for data in train_loader:
  print(data[0])

tensor([[0.7134, 0.5000, 0.5000],
        [0.9989, 0.5000, 0.5000]], dtype=torch.float64)
tensor([[0.7745, 0.5000, 0.5000],
        [0.5380, 0.5000, 0.5000]], dtype=torch.float64)
tensor([[0.5000, 0.5000, 0.9807],
        [0.5000, 0.5000, 0.4455]], dtype=torch.float64)
tensor([[0.5000, 0.3331, 0.5000],
        [0.5000, 0.0647, 0.5000]], dtype=torch.float64)


In [51]:
# Define the model, loss function, and optimizer
model = MeNN(4, 40, 4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model
for epoch in range(60):
    # Train on each batch
    for data in train_loader:
        for i in data:
          # Forward pass
          output = model(i[0])

          # Calculate loss
          loss = criterion(output, i[1])

          # Backward pass and update parameters
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

In [52]:
# Load the model
model.eval()

# Convert the input to a PyTorch tensor
input_tensor = torch.tensor([0.1, 0.0, 0.8, 0.9], dtype=torch.float64)

# Make a prediction
prediction = model(input_tensor)

print(prediction)


_, topk_indices = torch.topk(prediction, k=1)
action = topk_indices[0]
print(actions[action])

tensor([-0.8117,  0.1971, -0.9771,  0.4005], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
drink water


In [53]:
test_data = generate_dataset(4, [{'reverse': False}, {'reverse': False}, {'reverse': True}, {'reverse': False}], 30)
print(len(test_data))

240


In [54]:
hit = 0
total = 0
for data in test_data:
  input_tensor = torch.tensor(data[0], dtype=torch.float64)
  prediction = model(input_tensor) # input

  _, topk_indices = torch.topk(prediction, k=1)
  prediction_index = topk_indices[0]

  max_value = max(data[1])
  max_index = data[1].index(max_value)

  if max_index == prediction_index:
    hit+=1
  total+=1

print(f"Accuracy: %{100 * hit/total}")

Accuracy: %70.41666666666667
